In [69]:
import pandas as pd 
import table_cleaning_functions as tcf

In [70]:
# load raw data 
raw_df = pd.read_csv('../home/raw_data/gwas_biomarkers_revised.csv')
raw_df.head()

,Unnamed: 0,X,biomarker_ID,main_Xref,assessed_biomarker_entity,biomarker_status,best_biomarker_type,specimen_type,loinc_code,condition_name,assessed_entity_type,evidence_source,notes,system,DOID,Names,rsID
0,0,0,NaN,dbSNP:rs812481,rs812481 mutation in Leucine-rich repeat prote...,presence of,risk_biomarker,NaN,NaN,colorectal cancer (DOID:9256),gene,GWAS Catalog|https://www.ebi.ac.uk/gwas/search...,UPKB:Q5XWD3|GRCh38,NaN,9256.0,Leucine-rich repeat protein LRIG1,rs812481
1,1,1,NaN,dbSNP:rs73208120,"rs73208120 mutation in Nitric oxide synthase, ...",presence of,risk_biomarker,NaN,NaN,colorectal cancer (DOID:9256),gene,GWAS Catalog|https://www.ebi.ac.uk/gwas/search...,UPKB:P29475|GRCh38,NaN,9256.0,"Nitric oxide synthase, brain (EC 1.14.13.39) (...",rs73208120
2,2,2,NaN,dbSNP:rs11190164,rs11190164 mutation in (NKX2-3 - SLC25A28),presence of,risk_biomarker,NaN,NaN,colorectal cancer (DOID:9256),gene,GWAS Catalog|https://www.ebi.ac.uk/gwas/search...,GRCh38,NaN,9256.0,NaN,rs11190164
3,3,3,NaN,dbSNP:rs3184504,rs3184504 mutation in Ataxin-2 (ATXN2),presence of,risk_biomarker,NaN,NaN,colorectal cancer (DOID:9256),gene,GWAS Catalog|https://www.ebi.ac.uk/gwas/search...,UPKB:F8VQP2|GRCh38,NaN,9256.0,Ataxin-2,rs3184504
4,4,4,NaN,dbSNP:rs3184504,rs3184504 mutation in SH2B adapter protein 3 (...,presence of,risk_biomarker,NaN,NaN,colorectal cancer (DOID:9256),gene,GWAS Catalog|https://www.ebi.ac.uk/gwas/search...,UPKB:F5GYM4|GRCh38,NaN,9256.0,SH2B adapter protein 3,rs3184504


In [71]:
for col in raw_df.columns:
    raw_df[col] = tcf.strip_values(raw_df[col])

In [72]:
raw_df.isnull().sum()

Unnamed: 0                       0
X                                0
biomarker_ID                 10040
main_Xref                      146
assessed_biomarker_entity        0
biomarker_status                 0
best_biomarker_type              0
specimen_type                 9546
loinc_code                    9546
condition_name                   0
assessed_entity_type             0
evidence_source                  0
notes                            0
system                        9546
DOID                            78
Names                         5002
rsID                           146
dtype: int64

In [73]:
raw_df.shape

(10040, 17)

In [74]:
raw_df = raw_df.dropna(subset='rsID')
raw_df = raw_df.dropna(subset='DOID')
raw_df.shape

(9816, 17)

In [75]:
raw_df.isnull().sum()

Unnamed: 0                      0
X                               0
biomarker_ID                 9816
main_Xref                       0
assessed_biomarker_entity       0
biomarker_status                0
best_biomarker_type             0
specimen_type                9325
loinc_code                   9325
condition_name                  0
assessed_entity_type            0
evidence_source                 0
notes                           0
system                       9325
DOID                            0
Names                        4806
rsID                            0
dtype: int64

In [76]:
new_df = pd.DataFrame(columns = tcf.TSV_HEADERS)
new_df.head()

,biomarker_id,biomarker,assessed_biomarker_entity,assessed_biomarker_entity_id,assessed_entity_type,condition,condition_id,exposure_agent,exposure_agent_id,best_biomarker_role,specimen,specimen_id,loinc_code,evidence_source,evidence,tag


In [77]:
new_df['biomarker'] = raw_df['biomarker_status'] + ' ' + raw_df['assessed_biomarker_entity']
new_df['assessed_biomarker_entity'] = raw_df['assessed_biomarker_entity'].str.extract(r'\((?!.*-)([^\)]+)\)[^\(]*$')[0].str.strip()
new_df['assessed_biomarker_entity_id'] = raw_df['main_Xref']
new_df['assessed_entity_type'] = raw_df['assessed_entity_type']
new_df['best_biomarker_role'] = raw_df['best_biomarker_type'].str.replace('_biomarker', '')
new_df['specimen'] = raw_df['specimen_type'].str.findall(r'^(.*?)\(').str[0]
new_df['specimen_id'] = raw_df['specimen_type'].str.findall(r'\((.*?)\)$').str[0]
new_df.loc[new_df['specimen_id'] == 'UN:0000179', 'specimen_id'] = 'UN:0000178'
new_df['loinc_code'] = raw_df['loinc_code']
new_df['evidence_source'] = 'GWAS:' + new_df['assessed_biomarker_entity']
new_df['evidence'] = ''
new_df['tag'] = 'biomarker;assessed_biomarker_entity_ID;assessed_biomarker_entity;specimen;condition'

new_df['condition'] = raw_df['condition_name'].str.extract(r'^(.*?)\s*\(')
new_df['condition'] = tcf.strip_values(new_df['condition'])
condition_map = {
    'breast cancer': '1612',
    'prostate cancer': '10283',
    'lung cancer': '1324',
    'colorectal cancer': '9256',
    'skin cancer': '4159',
    'cancer': '162',
    'ovarian cancer': '2394',
    'cervical cancer': '4362',
    'pancreatic cancer': '1793',
    'endometrial cancer': '1380',
    'stomach cancer': '10534',
    'oropharynx cancer': '8557',
    'esophageal cancer': '5041',
    'brain cancer': '1319',
    'urinary bladder cancer': '11054',
    'liver cancer': '3571',
    'colon cancer': '219',
    'uterine cancer': '363',
    'kidney cancer': '263',
    'testicular cancer': '2998',
    'rectum cancer': '1993',
    'squamous cell carcinoma': '1749',
    'oral cavity cancer': '8618',
    'teratocarcinoma': '3305',
    'head and neck cancer': '11934',
    'larynx cancer': '2596',
    'central nervous system cancer': '3620',
    'bile duct cancer': '4606'
}
condition_drop_values = {'colorectal', 'esophageal'}
new_df['condition_id'] = 'DOID:' + new_df['condition'].str.lower().map(condition_map)
# drop values where condition is in the drop values set
new_df = new_df[~new_df['condition'].str.lower().isin(condition_drop_values)]

new_df.head()

,biomarker_id,biomarker,assessed_biomarker_entity,assessed_biomarker_entity_id,assessed_entity_type,condition,condition_id,exposure_agent,exposure_agent_id,best_biomarker_role,specimen,specimen_id,loinc_code,evidence_source,evidence,tag
0,NaN,presence of rs812481 mutation in Leucine-rich ...,LRIG1,dbSNP:rs812481,gene,colorectal cancer,DOID:9256,NaN,NaN,risk,NaN,NaN,NaN,GWAS:LRIG1,,biomarker;assessed_biomarker_entity_ID;assesse...
1,NaN,presence of rs73208120 mutation in Nitric oxid...,NOS1,dbSNP:rs73208120,gene,colorectal cancer,DOID:9256,NaN,NaN,risk,NaN,NaN,NaN,GWAS:NOS1,,biomarker;assessed_biomarker_entity_ID;assesse...
2,NaN,presence of rs11190164 mutation in (NKX2-3 - ...,NaN,dbSNP:rs11190164,gene,colorectal cancer,DOID:9256,NaN,NaN,risk,NaN,NaN,NaN,NaN,,biomarker;assessed_biomarker_entity_ID;assesse...
3,NaN,presence of rs3184504 mutation in Ataxin-2 (AT...,ATXN2,dbSNP:rs3184504,gene,colorectal cancer,DOID:9256,NaN,NaN,risk,NaN,NaN,NaN,GWAS:ATXN2,,biomarker;assessed_biomarker_entity_ID;assesse...
4,NaN,presence of rs3184504 mutation in SH2B adapter...,SH2B3,dbSNP:rs3184504,gene,colorectal cancer,DOID:9256,NaN,NaN,risk,NaN,NaN,NaN,GWAS:SH2B3,,biomarker;assessed_biomarker_entity_ID;assesse...


In [78]:
new_df.isnull().sum()

biomarker_id                    9779
biomarker                          0
assessed_biomarker_entity       3430
assessed_biomarker_entity_id       0
assessed_entity_type               0
condition                          0
condition_id                       0
exposure_agent                  9779
exposure_agent_id               9779
best_biomarker_role                0
specimen                        9289
specimen_id                     9289
loinc_code                      9289
evidence_source                 3430
evidence                           0
tag                                0
dtype: int64

In [79]:
new_df = new_df.dropna(subset='assessed_biomarker_entity')
new_df.isnull().sum()

biomarker_id                    6349
biomarker                          0
assessed_biomarker_entity          0
assessed_biomarker_entity_id       0
assessed_entity_type               0
condition                          0
condition_id                       0
exposure_agent                  6349
exposure_agent_id               6349
best_biomarker_role                0
specimen                        5881
specimen_id                     5881
loinc_code                      5881
evidence_source                    0
evidence                           0
tag                                0
dtype: int64

In [80]:
new_df.shape

(6349, 16)

In [81]:
new_df['biomarker'] = tcf.clean_parantheticals(new_df['biomarker'])
new_df['specimen_id'] = tcf.map_prefixes(new_df['specimen_id'])

In [82]:
new_df.to_csv('../home/cleaned_data/gwas_biomarkers.tsv', sep='\t', index=False)